
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
         print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:33<00:00,  2.98it/s]
/tmp/ipykernel_8318/523172613.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 2.0427963733673096
Epoch 10, Loss: 1.2911992073059082
Epoch 20, Loss: 0.9227162599563599
Epoch 30, Loss: 0.7650074362754822
Epoch 40, Loss: 0.6909422278404236
Epoch 50, Loss: 0.647700846195221
Epoch 60, Loss: 0.6177152991294861
Epoch 70, Loss: 0.5954135656356812
Epoch 80, Loss: 0.578117847442627
Epoch 90, Loss: 0.5642012357711792
Training complete!


## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

4. What would happen if we used directed edges instead of undirected edges for the random walks?
5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

### 1) What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?

If we increase the number of walks per node, I would hypothesize the embeddings to become strong since know there is more node neighborhoods being constructed, resulting in a stronger understanding of graph structure. This should then be reflected within the embeddings and the classifier should have a lower loss than before. 

To test this, I am simply going to increase the number of walks from 200 to 400 (double it) and see how the classifier performs

In [2]:
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=400, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [01:14<00:00,  2.70it/s]


Epoch 0, Loss: 1.9518219232559204
Epoch 10, Loss: 1.2408803701400757
Epoch 20, Loss: 0.8988996148109436
Epoch 30, Loss: 0.7474140524864197
Epoch 40, Loss: 0.6745575070381165
Epoch 50, Loss: 0.632602870464325
Epoch 60, Loss: 0.6045637726783752
Epoch 70, Loss: 0.583960235118866
Epoch 80, Loss: 0.5679291486740112
Epoch 90, Loss: 0.5549281239509583
Training complete!


Based off the results, you can tell that my hypothesis was indeed correct and the loss did decrease from .564 to .554. This is due to the reasons I stated above relating how with an increased number of walks there is also an increased understanding of the graph structure and in turn better node2vec embeddings leading to better classification accuracy by the neural network.

### 2) What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

If we reduced the walk length, this would influence the structural information captured by the embeddings due to the fact the there is a smaller magnitude of information being captured. For example, 200 walks of length 30 will yield 200 different node neighborhoods of 30 nodes, but 200 walks of length 10 will only yield 200 different node neighborhoods of 10. There is an upperbound for how high the walk length and num walks can go, but other than that decreasing the length of the walk will only capture less information resulting in weaker spectral embeddings. 

To test this, I will simply run the same model as the initial modeel with 200 num_walks, but change the walk_length to 10 instead of 30. We should see an increase in loss by making this change.

In [3]:
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=10, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
         print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:10<00:00,  9.51it/s]


Epoch 0, Loss: 2.054508924484253
Epoch 10, Loss: 1.3165380954742432
Epoch 20, Loss: 0.952644944190979
Epoch 30, Loss: 0.7927368879318237
Epoch 40, Loss: 0.7130237221717834
Epoch 50, Loss: 0.6647894978523254
Epoch 60, Loss: 0.6317962408065796
Epoch 70, Loss: 0.6075016856193542
Epoch 80, Loss: 0.5886807441711426
Epoch 90, Loss: 0.5735780000686646
Training complete!


As expected, my hypothesis was correct and we did in fact see an increase in loss from the initial model 0.564 to 0.574 (walk_length = 10). Like mentioned, this is due to less information being captured in the embeddings, leading to worse classifications by the network. 

### 3) What would happen if we used directed edges instead of undirected edges for the random walks?

To test this idea, I am changing the to_undirected variables in to_networkx to False, to make the graph directed. I want to see how the classifier performance is going to be affected by this change. 

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx

G = to_networkx(data, to_undirected=False)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
         print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:35<00:00,  2.84it/s]


Epoch 0, Loss: 2.0894391536712646
Epoch 10, Loss: 1.3139939308166504
Epoch 20, Loss: 0.9534384608268738
Epoch 30, Loss: 0.7869365215301514
Epoch 40, Loss: 0.7068240642547607
Epoch 50, Loss: 0.6608007550239563
Epoch 60, Loss: 0.6296052932739258
Epoch 70, Loss: 0.606698215007782
Epoch 80, Loss: 0.5887612104415894
Epoch 90, Loss: 0.5742022395133972
Training complete!


This model has the exact same parameters as the beggining model with walk length of 30 and num walks set to 2000, the only thing that changed was changing the graphs from undirected to directed. By making this change, you can see there has been a .01 increase in loss from .564 (undirected) to .574 (directed). This is in a large part due to the fact in how embeddings in node2vec are being created now that the graph has only directed edges. With undirected edges, the walk length of 30 was able to act freely as it new all nodes with edges are neighbors, but now since that constraint is not there, the graph becomes harder to read and more volatile with changes. Due to this, the embeddings made in node2vec are not as strong as the ones made when the graph was undirected. Since the classifier is all dependent on the quality of the embeddings, this clearly explains why there was an increase in loss when changing the graph from undirected to directed. 

### 4) What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?

If we added more features to the nodes, i.e. increasing each node's feature vector from 1433 to 2000, there wouldn't be any change on the node2vec embedding, hence no change on the overall classifier. This is in a larger part due to how the node2vec embeddings are being created. In the code we started with, each node is going to have 200 random walks of length 30 performed and while doing so information regarding the structure of the graph will be revealed. Nothing regarding the actual feature vectors of the graph is going to be affected. Hence, we don't even need the feature vector, in a way, as we are just looking for graph structure and node neighborhood and not any actual tangible information regarding the nodes. That would be used more in a message-passing algorithm. 

No code for this description at I believe it to pretty intuitive. If I were to change each feature vector, I would pad them all with 0s and I would expect any changes in the loss be due to probability.

### 5) What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?

If we were to run the same code on a different dataset with more classes the classifier performance would go down in accuracy. To start with creating the embeddings, nothing in terrms of the algorithm will change, however the resulting 64-dimensional embeddings will become less unique and as a result make the classifer perform worse. The embeddings will be less distinct due to there being more classes in the graph that may not all be covered the same amount during a random walk since it is due to probability. Since the classifiers are being trained off of the embeddings from node2vec, the more unique the embedddings are the better the classifier will be at predicting the class. Now, there are more classes, hence less robust embeddings causing a decrease in classifier performance.

No code was also shown for this due to not specifying in the question itself. However, it would not be very difficult to show. I would simply run the exact same model, with some minor tweaks like changing the output dimension. But, other than that it should be concrete enough to see that the embeddings will be worsened, although staying the same length, and the classifier performance will decrease (probably not significantly, but depends on how many new classes are added). 


### 6) What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?

If we were to enlarge the embedding dimensions from 64 to 128, there should be an increase in the model's performance. If there is any decrease, it is probably due to overfitting on the training data or the embedding picking up unnecessary information due to having more information than needed (depending on the graph itself). I would assume the training time to increase significantly. 

To test this, I will change the embedding dimension to 128, time how long it runs, and also check if the accuracy on the test increases. 

In [25]:
# 64 dimensional embedding

import time

start = time.time()
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
         print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")
end = time.time()
print(f"total time: {end - start}")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:36<00:00,  2.73it/s]


Epoch 0, Loss: 2.0100767612457275
Epoch 10, Loss: 1.283320665359497
Epoch 20, Loss: 0.9119169116020203
Epoch 30, Loss: 0.755591630935669
Epoch 40, Loss: 0.6841944456100464
Epoch 50, Loss: 0.6437552571296692
Epoch 60, Loss: 0.6167096495628357
Epoch 70, Loss: 0.5968500971794128
Epoch 80, Loss: 0.581413745880127
Epoch 90, Loss: 0.5688578486442566
Training complete!
total time: 470.3285357952118


In [24]:
# 128 dimensional embedding

import time

start = time.time()
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(128, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
         print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")
end = time.time()
print(f"total time: {end - start}")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:37<00:00,  2.70it/s]


Epoch 0, Loss: 1.9145219326019287
Epoch 10, Loss: 1.0649425983428955
Epoch 20, Loss: 0.7508973479270935
Epoch 30, Loss: 0.6398661136627197
Epoch 40, Loss: 0.5815190076828003
Epoch 50, Loss: 0.5418511629104614
Epoch 60, Loss: 0.5128682255744934
Epoch 70, Loss: 0.4906698763370514
Epoch 80, Loss: 0.4728759229183197
Epoch 90, Loss: 0.4580802917480469
Training complete!
total time: 486.28948187828064


The resuls are for the embeddings with 64 dimension vectors took 470 seconds to train and had a loss of .569 while the embeddings with 128 dimension vectors took 486 seconds and the loss was .458. Hence, my prediction that the accuracy of the classifier was correct, it was in fact a lot better. My training time prediction was still correct, but not nearly as much as I thought. Only 16 more seconds is nowhere near what I expected, so it definetely makes sense to use the higher dimensional embedding vectors as long as it is not overfit on training data!